<a href="https://colab.research.google.com/github/mtulow/dtc-de-zoomcamp2024/blob/main/dlt_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




In [1]:
questions = {
    1: 'What is the sum of the outputs of the generator for limit = 5?',
    2: 'What is the 13th number yielded by the generator?',
    3: 'Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.',
    4: 'Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.',
}
solutions = {
    1: None,
    2: None,
    3: None,
    4: None,
}

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit: int | float):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


#### Question 1

In [3]:
solutions[1] = sum(square_root_generator(limit=5))
print('Question 1:', questions[1])
print('Solution 1:', solutions[1])

Question 1: What is the sum of the outputs of the generator for limit = 5?
Solution 1: 8.382332347441762


### Question 2

In [4]:
solutions[2] = {idx: root for idx, root in enumerate(square_root_generator(20),1)}[13]
print('Question 2:', questions[2])
print('Solution 2:', solutions[2])

Question 2: What is the 13th number yielded by the generator?
Solution 2: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

### Install libraries

In [6]:
#Install the dependencies
%%capture
!pip install --quiet dlt[duckdb]

In [7]:
import dlt
import duckdb

#### Setup duckdb database

In [8]:
# Define the duckdb database to load to.
elt_pipeline = dlt.pipeline(destination='duckdb',
                            dataset_name='hw')

# Load the first generator to duckdb to create database table
info = elt_pipeline.run(people_1,
                        table_name='people_appended',
                        write_disposition='replace',
                        primary_key='id')

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

print()

# Load the first generator to duckdb to create database table
info = elt_pipeline.run(people_1,
                        table_name='people_merged',
                        write_disposition='replace',
                        primary_key='id')

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.66 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1709289438.7734098 is LOADED and contains no failed jobs

Pipeline dlt_colab_kernel_launcher load step completed in 0.56 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1709289440.03266 is LOADED and contains no failed jobs


In [9]:
# Connect to the duckdb database
conn = duckdb.connect(f"{elt_pipeline.pipeline_name}.duckdb")

# Display the people table
conn.sql(f"SET search_path = '{elt_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_appended     │
│ people_merged       │
└─────────────────────┘

#### Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [10]:
# we can load any generator to a table at the pipeline destnation as follows:
info = elt_pipeline.run(people_2(),
                        table_name="people_appended",
                        write_disposition="append",
                        primary_key="ID")

# Display database table
print("\n\n\n APPENDED `people_2` table to `people_1` table:")
df = conn.sql("SELECT * FROM people_appended").df()
display(df)




 APPENDED `people_2` table to `people_1` table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709289438.7734098,6LWyO5x8UrpEiw,None
1,2,Person_2,27,City_A,1709289438.7734098,A5zemqAlE0PG4A,None
2,3,Person_3,28,City_A,1709289438.7734098,Xjg13obNj1g7sw,None
3,4,Person_4,29,City_A,1709289438.7734098,eh3SkyjMBWtv1A,None
4,5,Person_5,30,City_A,1709289438.7734098,55Uu/mzvvJasoA,None
5,3,Person_3,33,City_B,1709289441.3741353,plOG6cnBHBKWlg,Job_3
6,4,Person_4,34,City_B,1709289441.3741353,rIZsE99Q+m9qRA,Job_4
7,5,Person_5,35,City_B,1709289441.3741353,OrsUHGjLQvFQ/A,Job_5
8,6,Person_6,36,City_B,1709289441.3741353,XNwwv0Z4fDR0mw,Job_6
9,7,Person_7,37,City_B,1709289441.3741353,8aVGV1MboCFjug,Job_7


In [11]:
solutions[3] = df['age'].sum()
print('Question 3:', questions[3])
print('Solution 3:', solutions[3])

Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
Solution 3: 353


#### Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [12]:
# we can load any generator to a table at the pipeline destnation as follows:
info = elt_pipeline.run(people_2(),
                        table_name="people_merged",
                        write_disposition="merge",
                        primary_key="ID")

# Display database table
print("\n\n\n MERGED `people_2` table to `people_1` table:")
df = conn.sql("SELECT * FROM people_merged").df()
display(df)




 MERGED `people_2` table to `people_1` table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709289440.03266,dGe2kK4EQQWFQw,None
1,2,Person_2,27,City_A,1709289440.03266,fNF9mSFKzP//Ow,None
2,8,Person_8,38,City_B,1709289442.7946708,AI+mn/1FpwjGKw,Job_8
3,7,Person_7,37,City_B,1709289442.7946708,GW/R3CJtBeUmOA,Job_7
4,4,Person_4,34,City_B,1709289442.7946708,CJb95NwM96QYKQ,Job_4
5,3,Person_3,33,City_B,1709289442.7946708,PCki5zEx23lTWg,Job_3
6,6,Person_6,36,City_B,1709289442.7946708,F/QotIrMrDo6rQ,Job_6
7,5,Person_5,35,City_B,1709289442.7946708,YTw4GNIpwPD8Fg,Job_5


In [13]:
solutions[4] = df['age'].sum()
print('Question 4:', questions[4])
print('Solution 4:', solutions[4])

Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.
Solution 4: 266


---
---

In [14]:
for idx in range(1,4+1):
    print()
    print(f'Question {idx}:', questions[idx])
    print(f'Solution {idx}:', solutions[idx])
    print()


Question 1: What is the sum of the outputs of the generator for limit = 5?
Solution 1: 8.382332347441762


Question 2: What is the 13th number yielded by the generator?
Solution 2: 3.605551275463989


Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
Solution 3: 353


Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.
Solution 4: 266



---
---

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

---
---
---